# Add active column on MoDaC uploaded existing DTC files (raw & test) (FRCE)

Written by Titli Sarkar (titli.sarkar@nih.gov) on March 3, 2023

MoDaC Data location (on FRCE): /mnt/projects/ATOM/sarkart4/MoDaC/NCI_DOE_Archive/ATOM/Safety_Screen_Targets/

In [21]:
#import sys
import atomsci.ddm.utils.data_curation_functions as dcf
import atomsci.ddm.utils.curate_data as curate_data, imp

In [22]:
import importlib as imp
import pandas as pd
import numpy as np

In [23]:
target = 'CYP2C9'

In [24]:
data_dir = "/mnt/projects/ATOM/sarkart4/MoDaC/NCI_DOE_Archive/ATOM/Safety_Screen_Targets/"+target.upper()+'-ampl-1.1.0/'
!ls $data_dir

CHEMBL25-CYP2C9_human_IC50_26Nov2019.txt
cyp2c9_chembl_testset_base_smiles_union.csv
cyp2c9_dtc_smiles.csv
cyp2c9_dtc_testset_base_smiles_union.csv
cyp2c9_excape_smiles.csv
cyp2c9_excape_testset_base_smiles_union.csv
cyp2c9_union_testset_base_smiles.csv
cyp2c9_union_trainset_base_smiles.csv
cyp2c9_union_trainset_base_smiles_train_valid_test_scaffold_41d32748-810d-4cd2-9cdb-124b77f657f8.csv
scaled_descriptors


In [25]:
# dtc_raw file contains PIC50 column
dtc_raw_df = pd.read_csv(data_dir + target.lower()+'_dtc_smiles.csv' ,sep=",",engine="python",error_bad_lines=False)
print(dtc_raw_df.shape)
dtc_raw_df.head(2)

(3198, 38)


/home/sarkart4/miniconda3/envs/ampl-1.4.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,compound_id,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,pubmed_id,...,assay_description,title,journal,doc_type,annotation_comments,PIC50,Unnamed: 0,CID,smiles,rdkit_smiles
0,CHEMBL1232461,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK525762A,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,24015967.0,...,Inhibition of human CYP2C9,Discovery of epigenetic regulator I-BET762: le...,J. Med. Chem.,PUBLICATION,NaN,4.481486,0,46943432,CCNC(=O)CC1C2=NN=C(N2C3=C(C=C(C=C3)OC)C(=N1)C4...,CCNC(=O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...
1,CHEMBL2158292,AADUJBXZMSDOOW-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,22039836.0,...,Inhibition of CYP2C9,Optimization of a potent class of arylamide co...,J. Med. Chem.,PUBLICATION,NaN,5.000000,1,11533340,C1CCC(=CC1)C2=C(C=CC(=C2)C3CCN(CC3)C(=O)C4=CC=...,N#Cc1cnc(C(=O)Nc2ccc(C3CCN(C(=O)c4cc[n+]([O-])...


In [26]:
dtc_raw_df.columns

Index(['compound_id', 'standard_inchi_key', 'compound_name', 'synonym',
       'target_id', 'target_pref_name', 'gene_names', 'wildtype_or_mutant',
       'mutation_info', 'pubmed_id', 'standard_type', 'standard_relation',
       'standard_value', 'standard_units', 'activity_comment',
       'ep_action_mode', 'assay_format', 'assaytype', 'assay_subtype',
       'inhibitor_type', 'detection_tech', 'assay_cell_line',
       'compound_concentration_value', 'compound_concentration_value_unit',
       'substrate_type', 'substrate_relation', 'substrate_value',
       'substrate_units', 'assay_description', 'title', 'journal', 'doc_type',
       'annotation_comments', 'PIC50', 'Unnamed: 0', 'CID', 'smiles',
       'rdkit_smiles'],
      dtype='object')

In [27]:
for v in dtc_raw_df['standard_units'].unique() :  
    t=dtc_raw_df[dtc_raw_df['standard_units']==v]
    print(v,t.shape)

NM (3198, 38)


In [28]:
dtc_raw_df['PIC50']

0       4.481486
1       5.000000
2       5.000000
3       5.292430
4       5.000000
          ...   
3193    4.301030
3194    4.301030
3195    4.500000
3196    4.795880
3197    6.124939
Name: PIC50, Length: 3198, dtype: float64

### Apply basic AMPL data curation 

In [29]:
imp.reload(dcf)
data=dtc_raw_df
column = 'PIC50'
tolerance = 10
list_bad_duplicates = 'Yes'
max_std = 1
compound_id = 'compound_id'
smiles_col = 'rdkit_smiles'
check_df = curate_data.average_and_remove_duplicates(column, tolerance, 
                                                       list_bad_duplicates, 
                                                       data, max_std, 
                                                       compound_id=compound_id, 
                                                       smiles_col=smiles_col)
# note: checked that this is already curated
[check_df.shape]
check_df.head()
# before (6050, 42)
# after (5004, 42) 
# Which means MoDaC uploaded 'raw' data have duplicates! 

Bad duplicates removed from dataset
Dataframe size (3153, 42)
List of 'bad' duplicates removed
        compound_id     PIC50  VALUE_NUM_mean   Perc_Var  VALUE_NUM_std
2760  CHEMBL1079256  4.000000        4.500000  11.111111       0.707107
2759  CHEMBL1079256  5.000000        4.500000  11.111111       0.707107
2065     CHEMBL1109  5.376751        6.250422  13.977791       0.817252
2067     CHEMBL1109  5.376751        6.250422  13.977791       0.817252
2068     CHEMBL1109  7.096910        6.250422  13.542902       0.817252
2061     CHEMBL1109  3.497573        6.250422  44.042608       0.817252
611    CHEMBL129134  5.744727        4.244864  35.333613       2.121128
612    CHEMBL129134  2.745000        4.244864  35.333613       2.121128
1567      CHEMBL163  7.795880        6.552842  18.969450       1.757921
1566      CHEMBL163  5.309804        6.552842  18.969450       1.757921
1676    CHEMBL16596  9.301030        7.801030  19.228230       2.121320
1677    CHEMBL16596  6.301030        7.80

,compound_id,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,pubmed_id,...,annotation_comments,PIC50,Unnamed: 0,CID,smiles,rdkit_smiles,VALUE_NUM_mean,VALUE_NUM_std,Perc_Var,Remove_BadDuplicate
0,CHEMBL1232461,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK525762A,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,24015967.0,...,NaN,4.481486,0,46943432,CCNC(=O)CC1C2=NN=C(N2C3=C(C=C(C=C3)OC)C(=N1)C4...,CCNC(=O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...,4.481486,NaN,0.0,0
1,CHEMBL2158292,AADUJBXZMSDOOW-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,22039836.0,...,NaN,5.000000,1,11533340,C1CCC(=CC1)C2=C(C=CC(=C2)C3CCN(CC3)C(=O)C4=CC=...,N#Cc1cnc(C(=O)Nc2ccc(C3CCN(C(=O)c4cc[n+]([O-])...,5.000000,NaN,0.0,0
2,CHEMBL1086377,AATCBLYHOUOCTO-UHFFFAOYSA-N,KU-0060648,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,23855836.0,...,NaN,5.000000,2,11964036,CCN1CCN(CC1)CC(=O)NC2=C3C4=CC=CC=C4SC3=C(C=C2)...,CCN1CCN(CC(=O)Nc2ccc(-c3cccc4c(=O)cc(N5CCOCC5)...,5.000000,NaN,0.0,0
3,CHEMBL2017279,AAVAYKQPRIQKRX-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,22386529.0,...,NaN,5.292430,3,70685377,CC1=C(C(=NO1)C)C2=C(C=C3C(=C2)N=CC(=C3NC4=CC=C...,COc1cc2c(Nc3ccccc3N3CCOCC3)c(C(N)=O)cnc2cc1-c1...,5.292430,NaN,0.0,0
4,CHEMBL490153,ABACVOXFUHDKNZ-UHFFFAOYSA-N,A-867744,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,19419141.0,...,NaN,5.000000,4,23642319,CCC(=O)C1=C(N(C(=C1)C2=CC=C(C=C2)Cl)C3=CC=C(C=...,CCC(=O)c1cc(-c2ccc(Cl)cc2)n(-c2ccc(S(N)(=O)=O)...,5.000000,NaN,0.0,0


### Add 'active' column  

In [30]:
imp.reload(curate_data)

tolerance=10
column='PIC50'; #'standard_value'
list_bad_duplicates='No'
max_std=1
list_bad_duplicates='Yes'
data=check_df
print("before",data.shape)

temp_df=curate_data.aggregate_assay_data(data, value_col=column, output_value_col=None,
                         label_actives=True,
                         active_thresh=6, # val > 6 -> 'active' (drug-like compounds) # can be - None(default)| This creates 'active' column based on pIC50 value. Rule used here: pIC50 >=6 'active', labelled as 0, else 1
                         id_col='standard_inchi_key', smiles_col='rdkit_smiles', relation_col='standard_relation')

# Remove inf in curated_df
temp_df = temp_df[~temp_df.isin([np.inf]).any(1)]
print("after",temp_df.shape)
temp_df.head()

before (2824, 42)
after (2824, 5)


,compound_id,base_rdkit_smiles,relation,PIC50,active
0,RIFYAWGYIHDAHH-UHFFFAOYSA-N,O=C(Nc1ccc(Cl)cc1)N1CCCC(CCO)(c2ccccc2)C1,<,5.000000,0
1,ZTTKEBYSXUCBSE-VSBZUFFNSA-N,COc1ccc2c(c1)C1CC1(C(=O)N1C3CCC1CN(C)C3)Cn1c-2...,>,4.397940,0
2,OGUUDCBPGBXZHI-KRWDZBQOSA-N,CS(=O)(=O)N1CCC(N(Cc2ccccc2Br)c2ccc(C#N)c(Cl)c...,,6.161151,1
3,QGUGSIVNZPWVQH-UHFFFAOYSA-N,Cc1ccc(-n2nc(C(C)(C)C)cc2NC(=O)Nc2ccc(-n3cnnn3...,>,5.000000,0
4,YREXUDMRKQYWKW-UHFFFAOYSA-N,COCCOCn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)cc3)c...,,5.283997,0


In [31]:
# Add 'active' column to original data and save file to FRCE
merged = pd.merge(left=dtc_raw_df, right=temp_df, how='left', left_on='rdkit_smiles', right_on='base_rdkit_smiles')
merged = merged.drop(columns=['compound_id_y', 'PIC50_y', 'base_rdkit_smiles'])
print(merged.shape)
merged.head(5)

(3198, 40)


,compound_id_x,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,pubmed_id,...,journal,doc_type,annotation_comments,PIC50_x,Unnamed: 0,CID,smiles,rdkit_smiles,relation,active
0,CHEMBL1232461,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK525762A,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,24015967.0,...,J. Med. Chem.,PUBLICATION,NaN,4.481486,0,46943432,CCNC(=O)CC1C2=NN=C(N2C3=C(C=C(C=C3)OC)C(=N1)C4...,CCNC(=O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...,,0.0
1,CHEMBL2158292,AADUJBXZMSDOOW-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,22039836.0,...,J. Med. Chem.,PUBLICATION,NaN,5.000000,1,11533340,C1CCC(=CC1)C2=C(C=CC(=C2)C3CCN(CC3)C(=O)C4=CC=...,N#Cc1cnc(C(=O)Nc2ccc(C3CCN(C(=O)c4cc[n+]([O-])...,>,0.0
2,CHEMBL1086377,AATCBLYHOUOCTO-UHFFFAOYSA-N,KU-0060648,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,23855836.0,...,J. Med. Chem.,PUBLICATION,NaN,5.000000,2,11964036,CCN1CCN(CC1)CC(=O)NC2=C3C4=CC=CC=C4SC3=C(C=C2)...,CCN1CCN(CC(=O)Nc2ccc(-c3cccc4c(=O)cc(N5CCOCC5)...,>,0.0
3,CHEMBL2017279,AAVAYKQPRIQKRX-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,22386529.0,...,Bioorg. Med. Chem. Lett.,PUBLICATION,NaN,5.292430,3,70685377,CC1=C(C(=NO1)C)C2=C(C=C3C(=C2)N=CC(=C3NC4=CC=C...,COc1cc2c(Nc3ccccc3N3CCOCC3)c(C(N)=O)cnc2cc1-c1...,,0.0
4,CHEMBL490153,ABACVOXFUHDKNZ-UHFFFAOYSA-N,A-867744,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,19419141.0,...,J. Med. Chem.,PUBLICATION,NaN,5.000000,4,23642319,CCC(=O)C1=C(N(C(=C1)C2=CC=C(C=C2)Cl)C3=CC=C(C=...,CCC(=O)c1cc(-c2ccc(Cl)cc2)n(-c2ccc(S(N)(=O)=O)...,>,0.0


In [32]:
merged.active.value_counts()

0.0    2885
1.0     275
Name: active, dtype: int64

In [33]:
merged.to_csv(data_dir +target.lower()+'_dtc_smiles_active.csv')

# Do The same for other datasets 

***Note: 'active' column was marked in dtc_testset file uploaded on MoDaC, but we do not know how it was created, so we are follwoing same rule uniformly for all dataset we are using.***

**The basic functions to be used:**

**cd.average_and_remove_duplicates()**

**cd.aggregate_assay_data()**


# Add 'active' to dtc_test

### Just find the correct marking of compounds for 'active' column from dtc_raw_active.csv based on 'rdkit_smiles' column 


In [34]:
dtc_raw_active = pd.read_csv(data_dir + target.lower()+'_dtc_smiles_active.csv' ,sep=",")
print(dtc_raw_active.shape)
dtc_raw_active.head(2)

(3198, 41)


,Unnamed: 0,compound_id_x,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,...,journal,doc_type,annotation_comments,PIC50_x,Unnamed: 0.1,CID,smiles,rdkit_smiles,relation,active
0,0,CHEMBL1232461,AAAQFGUYHFJNHI-SFHVURJKSA-N,GSK525762A,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,...,J. Med. Chem.,PUBLICATION,NaN,4.481486,0,46943432,CCNC(=O)CC1C2=NN=C(N2C3=C(C=C(C=C3)OC)C(=N1)C4...,CCNC(=O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...,NaN,0.0
1,1,CHEMBL2158292,AADUJBXZMSDOOW-UHFFFAOYSA-N,NaN,NaN,P11712,CYTOCHROME P450 2C9,CYP2C9,NaN,NaN,...,J. Med. Chem.,PUBLICATION,NaN,5.000000,1,11533340,C1CCC(=CC1)C2=C(C=CC(=C2)C3CCN(CC3)C(=O)C4=CC=...,N#Cc1cnc(C(=O)Nc2ccc(C3CCN(C(=O)c4cc[n+]([O-])...,>,0.0


In [35]:
dtc_test_df = pd.read_csv(data_dir + target.lower()+'_dtc_testset_base_smiles_union.csv' ,sep=",")
dtc_test_df = dtc_test_df.drop(columns = ['active'])
print(dtc_test_df.shape) # Note: wrong 'active' column, will be marked correctly in next step
dtc_test_df.head(2)

(350, 4)


,PIC50,base_rdkit_smiles,compound_id,relation
0,4.522879,CN(C1CCc2c(c3ccncc3n2CC(=O)O)C1)S(=O)(=O)c1ccc...,UHMQYWPLWZKMOS-CQSZACIVSA-N,>
1,5.000000,Cc1ccc(-n2nc(C(C)(C)C)cc2NC(=O)Nc2ccc(-c3n[nH]...,LCGWKKYPAXXXPK-UHFFFAOYSA-N,>


In [36]:
# Add 'active' column and save file to FRCE
#merged = pd.merge(left=dtc_test_df, right=dtc_raw_active, how='left', left_on='base_rdkit_smiles', right_on='rdkit_smiles')
merged = dtc_test_df.merge(dtc_raw_active[['rdkit_smiles', 'active']], how = 'left', 
                           left_on = 'base_rdkit_smiles', right_on = 'rdkit_smiles').drop(columns = ['rdkit_smiles'])
print(merged.shape)
merged.head(2)

(386, 5)


,PIC50,base_rdkit_smiles,compound_id,relation,active
0,4.522879,CN(C1CCc2c(c3ccncc3n2CC(=O)O)C1)S(=O)(=O)c1ccc...,UHMQYWPLWZKMOS-CQSZACIVSA-N,>,0.0
1,5.000000,Cc1ccc(-n2nc(C(C)(C)C)cc2NC(=O)Nc2ccc(-c3n[nH]...,LCGWKKYPAXXXPK-UHFFFAOYSA-N,>,0.0


In [37]:
merged.active.value_counts()

0.0    351
1.0     31
Name: active, dtype: int64

In [38]:
merged.to_csv(data_dir +target.lower()+'_dtc_testset_base_smiles_union_active.csv')